# Ejercicio adicional de fin de semana: semana 3

Utlizando repositorios closed source (OpenAI, claude, gemini) u open source (HuggingFace), realizar:

Utilizar modelos para generar datos sintéticos
Utilizar una variedad de modelos y prompts para diversos resultados
Crear una interfaz en Gradio UI para tu producto.

In [1]:
!pip install -q-U bitsandbytes accelerate


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --


In [2]:
import os
import json
import torch
import gradio as gr
import getpass
from dotenv import load_dotenv
from openai import OpenAI
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
from huggingface_hub import login

In [ ]:
# HuggingFace login

# hf_token = userdata.get('HF_TOKEN')
# login(hf_token, add_to_git_credential=True)
login()

In [13]:
# OpenAI login
api_key = getpass.getpass("Introduce tu OpenAI API Key y pulsa Enter: ")
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=api_key)

In [5]:
# # Inicialización
# load_dotenv()

# openai_api_key = os.getenv('OPENAI_API_KEY')
# if openai_api_key:
#     print(f"OpenAI API Key configurada correctamente")
# else:
#     print("OpenAI API Key sin configurar")

# Modelos
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
OPENAI = "gpt-4o-mini"

In [6]:
# Quantization Config - Esto nos permite cargar el modelo en la memoria y utilizar menos memoria.
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [7]:
SYSTEM_PROMPT = """
Eres un experto en ciencia de datos actuariales y normativa de seguros (GDPR/Solvencia II).
Tu tarea es generar datos sintéticos realistas para pruebas de software.

REGLAS CRÍTICAS:
1. Formato: Devuelve ÚNICAMENTE código JSON puro. No incluyas explicaciones.
2. Realismo: Las fechas de siniestro deben ser posteriores a la fecha de inicio de la póliza.
3. Coherencia: Si el cliente tiene un historial de alta siniestralidad, la prima (premium) debe ser proporcionalmente más alta.
4. Anonimización: Usa nombres ficticios y nunca generes números de identificación realistas (DNI/SSN).
"""
SYSTEM_PROMPT += " El resultado debe ser una LISTA (array) de objetos JSON."

In [9]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto",
                                             quantization_config=quant_config)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [14]:
def generar_datos_seguros(perfil, cantidad, modelo_nombre):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Genera exactamente {int(cantidad)} registros de tipo: {perfil}"}
    ]
    
    if modelo_nombre == LLAMA:
        # 1. Aplicar el template del chat
        inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")
        
        # 2. Generar con límites claros para evitar bucles infinitos
        outputs = model.generate(
            inputs, 
            max_new_tokens=2048, # Suficiente para varios JSONs
            temperature=0.8,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        
        # 3. Decodificar solo la parte nueva (la respuesta)
        response_text = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
        return response_text
    
    else:
        # Lógica para OpenAI
        response = client.chat.completions.create(
            model=modelo_nombre,
            messages=messages,
            temperature=0.8
        )
        return response.choices[0].message.content

In [15]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🛡️ Actuarial Data Generator")
    with gr.Row():
        with gr.Column():
            perfil = gr.Dropdown(["estandar", "fraude", "siniestro_vivienda"], label="Perfil de Datos")
            cantidad = gr.Slider(1, 10, step=1, label="Cantidad de Registros")
            selector_modelo = gr.Radio([OPENAI, LLAMA], label="Modelo", value=OPENAI)
            btn = gr.Button("Generar JSON")
        
        output_box = gr.Code(language="json", label="Resultado")

    btn.click(
        fn=generar_datos_seguros, 
        inputs=[perfil, cantidad, selector_modelo], 
        outputs=output_box
    )

# share=True es vital para ver la UI desde VS Code/Local
demo.launch(share=True, debug=True)

/tmp/ipython-input-2107624074.py:1: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://87f95a6602ab689736.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://87f95a6602ab689736.gradio.live
